## Изучите возможности инструментов

В каждом инструменте есть полно всяких параметров, которые можно подкручивать, чтобы получить нужный результат. Посмотрите в официальную документацию, чтобы узнать как. Для VW вся документация находится в Github - https://github.com/VowpalWabbit/vowpal_wabbit/wiki .

Все эти параметры можно использовать - начиная от указания весов для различных классов, заканчивая нейронными сетями в VW.

Одно из полезных знаний про VW - помимо задачи регрессии он умеет решать и задачу классификации, если использовать соответствующую фукнцию потерь. Важно - для бинарной классификации необходимо использовать классы 1 и -1 вместо классических 1 и 0.

Ниже пример того, как можно обучать логрег на VW.

In [1]:
%%local

! wget http://hunch.net/~vw/rcv1.tar.gz -O rcv1.tar.gz
! tar -zxvf rcv1.tar.gz
! cat rcv1/rcv1.train.vw.gz | gunzip | sed -e 's/^0/-1/' > logistic.train.vw
! cat rcv1/rcv1.test.vw.gz | gunzip | sed -e 's/^0/-1/' > logistic.test.vw

--2021-02-24 20:57:34--  http://hunch.net/~vw/rcv1.tar.gz
Resolving hunch.net (hunch.net)... 188.138.121.136
Connecting to hunch.net (hunch.net)|188.138.121.136|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://hunch.net/~vw/rcv1.tar.gz [following]
--2021-02-24 20:57:35--  https://hunch.net/~vw/rcv1.tar.gz
Connecting to hunch.net (hunch.net)|188.138.121.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456931808 (436M) [application/x-gzip]
Saving to: ‘rcv1.tar.gz’

rcv1.tar.gz         100%[===================>] 435.76M  11.2MB/s    in 45s     

2021-02-24 20:58:20 (9.67 MB/s) - ‘rcv1.tar.gz’ saved [456931808/456931808]

rcv1/
rcv1/rcv1.train.vw.gz
rcv1/vw_process
rcv1/rcv1.test.vw.gz


In [7]:
%%local 

! head -n 3 logistic.train.vw

1 |f 13:3.9656971e-02 24:3.4781646e-02 69:4.6296168e-02 85:6.1853945e-02 140:3.2349996e-02 156:1.0290844e-01 175:6.8493910e-02 188:2.8366476e-02 229:7.4871540e-02 230:9.1505975e-02 234:5.4200061e-02 236:4.4855952e-02 238:5.3422898e-02 387:1.4059304e-01 394:7.5131744e-02 433:1.1118756e-01 434:1.2540409e-01 438:6.5452829e-02 465:2.2644201e-01 468:8.5926279e-02 518:1.0214076e-01 534:9.4191484e-02 613:7.0990764e-02 646:8.7701865e-02 660:7.2289191e-02 709:9.0660661e-02 752:1.0580081e-01 757:6.7965068e-02 812:2.2685185e-01 932:6.8250686e-02 1028:4.8203137e-02 1122:1.2381379e-01 1160:1.3038123e-01 1189:7.1542501e-02 1530:9.2655659e-02 1664:6.5160148e-02 1865:8.5823394e-02 2524:1.6407280e-01 2525:1.1528353e-01 2526:9.7131468e-02 2536:5.7415009e-01 2543:1.4978983e-01 2848:1.0446861e-01 3370:9.2423186e-02 3960:1.5554591e-01 7052:1.2632671e-01 16893:1.9762035e-01 24036:3.2674628e-01 24303:2.2660980e-01
-1 |f 9:8.5609287e-02 14:2.9904654e-02 19:6.1031535e-02 20:2.1757640e-02 24:1.3484491e-02 39:5.

In [2]:
%%local

! vw logistic.train.vw \
    --final_regressor logistic.model.bin \
    --loss_function logistic \
    --learning_rate 0.1 \
    --passes 10 \
    --cache -k

final_regressor = logistic.model.bin
Num weight bits = 18
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = logistic.train.vw.cache
Reading datafile = logistic.train.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.693147 0.693147            1            1.0   1.0000   0.0000       50
0.716544 0.739940            2            2.0  -1.0000   0.0915      103
0.694031 0.671518            4            4.0  -1.0000  -0.0699      134
0.662117 0.630204            8            8.0  -1.0000  -0.4382      145
0.660671 0.659225           16           16.0   1.0000  -0.5270      142
0.648020 0.635369           32           32.0   1.0000  -0.3201       69
0.641225 0.634430           64           64.0  -1.0000  -0.0681       33
0.653087 0.664949          128          128.0  -1.0000  -0.5420       29
0.626895 0

Если требуется предсказание конкретных классов, то нужно использовать параметр `--binary`

In [9]:
%%local

! vw logistic.test.vw \
    --testonly \
    --binary \
    --initial_regressor logistic.model.bin \
    --predictions logistic.predictions.txt

only testing
predictions = logistic.predictions.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = logistic.test.vw
num sources = 1
Enabled reductions: gd, scorer, binary
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0  -1.0000  -1.0000       40
0.000000 0.000000            2            2.0   1.0000   1.0000       74
0.000000 0.000000            4            4.0  -1.0000  -1.0000       43
0.000000 0.000000            8            8.0  -1.0000  -1.0000       41
0.000000 0.000000           16           16.0   1.0000   1.0000       33
0.031250 0.062500           32           32.0  -1.0000  -1.0000      209
0.046875 0.062500           64           64.0   1.0000   1.0000       24
0.046875 0.046875          128          128.0   1.0000   1.0000       44
0.085938 0.125000          256          256

In [10]:
%%local

! head logistic.predictions.txt

-1
1
-1
-1
-1
-1
-1
-1
-1
-1


Чтобы предсказывать вероятности, необходимо указывать `--link=logistic`

In [11]:
%%local

! vw logistic.test.vw \
    --testonly \
    --link=logistic \
    --initial_regressor logistic.model.bin \
    --predictions logistic.predictions.txt

only testing
predictions = logistic.predictions.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = logistic.test.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
3.896363 3.896363            1            1.0  -1.0000   0.0486       40
12.191053 20.485743            2            2.0   1.0000   0.9960       74
18.968879 25.746705            4            4.0  -1.0000   0.0009       43
17.657918 16.346958            8            8.0  -1.0000   0.0211       41
15.201921 12.745923           16           16.0   1.0000   0.9852       33
16.556192 17.910464           32           32.0  -1.0000   0.0018      209
16.887389 17.218586           64           64.0   1.0000   0.9028       24
17.998370 19.109352          128          128.0   1.0000   0.9894       44
17.750270 17.502170          256     

In [12]:
%%local

! head logistic.predictions.txt 

0.048618
0.996034
0.006700
0.000950
0.000761
0.008218
0.039817
0.021137
0.097306
0.193052


## Поиграйтесь с SQL

SQL - полезная штука и очень хорошо было бы поднять свое знание этой технологии.

При сборке датасета в особенности важно следить за правильным использованием джойнов - их есть несколько видов, почитайте про них например на wiki - https://ru.wikipedia.org/wiki/Join_(SQL) .

## Тюньте кластер под ваши нужды

У спарка полно параметров. Их все можно подстраивать под себя.

Например в домашке может возникнуть проблема, когда у спарка не получится сджойнить таблицы через метод бродкаст. Это можно поправить или выдав больше памяти для воркера или отключив вообще этот метод. Пример как это сделать смотри ниже

```
%%configure -f
{
    "executorMemory": "20000M", 
    "executorCores": 4, 
    "numExecutors":5,
    "conf": {
        "spark.sql.autoBroadcastJoinThreshold": -1
    }
}
```

Более подробный гайд по перформансу можно прочитать в официальной документации - https://spark.apache.org/docs/latest/sql-performance-tuning.html


Помимо этого вы можете просто закинуть больше машин в кластер. Но здесь есть два нюанса

* Вы будете тратить больше денег. Контролируйте свой баланс и используйте мощности только если уже уверены в своих алгоритмах. Тестировать гипотезы лучше на небольших кластерах.
* Возможно у вас есть ограничение на количество ядер, которые вы можете арендовать. Это можно решить, отправив запрос на увеличение квоты. **ВАЖНО** - **эти запросы обрабатывают реальные люди, а значит это нужно сделать заранее, чтобы вы успели до дедлайна**.